# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [ ]:
train_dir = "/media/alper/B00A1E730A1E36B6/Datasets/fire_smoke_neutral/FIRE-SMOKE-DATASET/Train"
valid_dir = "/media/alper/B00A1E730A1E36B6/Datasets/fire_smoke_neutral/FIRE-SMOKE-DATASET/Test"

In [ ]:
train_data = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=45,
    fill_mode="nearest"
)

valid_data = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_gen = train_data.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    class_mode="categorical",
    batch_size=16
)

valid_gen = valid_data.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    class_mode="categorical",
    batch_size=16
)

# Model

In [ ]:
model = Sequential()
model.add(Conv2D(96, (11, 11), strides=(4, 4), activation="relu", input_shape=(224, 224, 3)))
model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(256, (5, 5), activation="relu"))
model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(512, (5, 5), activation="relu"))
model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(2048, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(2, activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=1e-3), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_gen, steps_per_epoch=25, epochs=50, validation_data=valid_gen, validation_steps=25)

In [ ]:
plt.figure()
plt.plot(history.history["accuracy"], label="train accuracy")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history["loss"], label="train loss")
plt.legend()
plt.show()

In [ ]:
model.save("fire_model.h5")